In [1]:
import sys, importlib
# "../" to go back one director
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
import os
# import torch.nn as nn
# from Modules.Utils.Imports import *
# from Modules.Utils.JTNPDESolver import *
# from Modules.Utils.Gradient import Gradient
# from Modules.Utils.ModelWrapper import ModelWrapper
# from Modules.Models.BuildSurfaceFitter import *
# from torch.autograd import Variable
# import Modules.Utils.PDESolver as PDESolver
# import DataFormatter as DF

import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
import scipy.io
from scipy import integrate, interpolate
from scipy.sparse import spdiags
from scipy import sparse
import bisect
import time

from scipy.optimize import linprog
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint

In [2]:
def constraints_func(params, num_PDE):
    ''' constraints function for the weights
    '''
    return 1.0 - sum(params[2*num_PDE:])

def PDE_Error(RHS, PDE_sim, u0, x, t, params, K, num_PDE, u_data):
    ''' return the sum of squares error when using traditional inverse problem approach
    '''
    u_sim = PDE_sim(RHS,u0,x,t,params,K,num_PDE)
    u_sim = u_sim.reshape((num_PDE,len(x),len(t)))
    u_sim = np.sum(u_sim, axis=0)
    
    residual = (u_sim - u_data)**2

    return sum(sum(residual))

def PDE_RHS(t,u,x,params,K,num_PDE):
    
    ''' returns a RHS of the RDE/PDE models
    '''
    
    # Construct the finite difference matrix
    n = len(x)
    e = np.ones((n,))
    A = spdiags([e,-2*e,e], [-1,0,1], n, n)
    A = A.todense()
    A[0,1] = 2
    A[-1,-2] = 2
    dx = x[1] - x[0]
    
    # Get parameter values
    Ds = params[0:num_PDE]
    RhoS = params[num_PDE:2*num_PDE]
    
    # Compute the aggregated population density
    u = u.reshape((num_PDE,len(x)))
    u_sum = np.sum(u, axis=0)

    # Compute the RHS for each subpopulation
    rhs = np.zeros_like(u)
    for i in np.arange(num_PDE):
        rhs[i,:] = Ds[i]*A.dot(u[i,:])/dx**2 + RhoS[i]*u[i,:]*(1 - u_sum/K)
    
    #
    rhs = rhs.reshape((1,-1))[0]
    
    return rhs

def PDE_sim(RHS,u0,x,t,params,K,num_PDE):
    
    #
    t_min = np.min(t)
    t_max = np.max(t)
    t_data = t
    
    # Compute the subpopulations initial condition
    IC = np.zeros((num_PDE,len(x)))
    weights = params[2*num_PDE:]
    for i in np.arange(num_PDE):
        IC[i,:] = u0*weights[i]
        
    IC = IC.reshape((1,-1))[0]
    
    #make RHS a function of t,y
    def RHS_ty(t,y):
        return RHS(t,y,x,params,K,num_PDE)
    
    # Solve 
    sol = integrate.solve_ivp(RHS_ty, t_span=[t_min, t_max], y0=IC, method='RK45', t_eval = t)
    
    u = sol.y
    
    return u

In [ ]:
# Load generated data
path = os.getcwd()+'/NPY_Data/data_wider_2P_E2.npy'
data = np.load(path, allow_pickle=True).item()

x = data['x']
t = data['t']
t = t[:26]
U = data['u_noise']
U = U[:,:26]
# t = t/
X, T = np.meshgrid(x, t, indexing='ij')

shape = U.shape

# flatten for MLP
inputs = np.concatenate([X.reshape(-1)[:, None],
                         T.reshape(-1)[:, None]], axis=1)
outputs = U.reshape(-1)[:, None]

X = inputs[:,0]
T = inputs[:,1]

U = outputs.reshape(shape)

x = np.unique(X)
t = np.unique(T)
u0 = U[:,0]

# simulate PDE
RHS = PDE_RHS
num_PDE = 2

# Setting up the parameters bounds
D_min = 0
D_max = 0.12
rho_min = 0
rho_max = 12.0
K = 1

num_run = 20
pmf_error = 1e16
best_results = {}

folder_path = 'PDE_Fitting_Results_2P_E2/PDEFitting_N01_'+str(num_PDE)+'P'
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    
save_file = 'PDE_Fitting_Results_2P_E2/PDEFitting_N01_'+str(num_PDE)+'P/PDEFitting_N01.npy'

for i in np.arange(num_run):
    
    save_run = 'PDE_Fitting_Results_2P_E2/PDEFitting_N01_'+str(num_PDE)+'P/PDEFitting_N01_R'+str(i+1).zfill(2)+'.npy'
    
    if not os.path.exists(save_run):
    
        np.random.seed(i)
        
        # initialize weights
        params0 = np.random.rand(num_PDE*3)
        params0[0:num_PDE] = params0[0:num_PDE]*(D_max - D_min) + D_min
        params0[num_PDE:2*num_PDE] = params0[num_PDE:2*num_PDE]*(rho_max - rho_min) + rho_min
        params0[2*num_PDE:] = params0[2*num_PDE:]/sum(params0[2*num_PDE:])

        # Setting up bounds
#        lb = np.zeros((num_PDE*3, 1))
#        lb[num_PDE:2*num_PDE] = K_min
#        ub = np.ones((num_PDE*3, 1))
#        ub[0:num_PDE] = D_max
#
#        bnds = Bounds(lb, ub)
        lb = [0, 0, 0, 0, 0, 0]
        ub = [D_max, D_max, rho_max, rho_max, 1, 1]
        bnds = tuple(list(zip(lb, ub)))
        
        # Setting up constraints for weights
        def constraints(params):
            return constraints_func(params, num_PDE)

        # try a different way to write constraints
        cons = {'type':'eq', 'fun': constraints}
        cons = [cons]
        
        #make a wrapper objective function
        def objective(params):
            return PDE_Error(RHS, PDE_sim, u0, x, t, params, K, num_PDE, U)
        
        t0 = time.time()
        
        # Find the optimal parameters
        sol = minimize(objective, params0, bounds = bnds, constraints=cons)
        elapsed_time = time.time() - t0
        
        optimized_fval = sol.fun
        optimized_params = sol.x
        
        results = {}
        results['params0'] = params0
        results['optimized_params'] = optimized_params
        results['error'] = optimized_fval
        results['elapsed_time'] = elapsed_time
        
        np.save(save_run, results)
        
#        if optimized_fval < pmf_error:
#            pmf_error = optimized_fval
#            best_params0 = params0
#            best_params  = optimized_params
#
#            best_results['best_error'] = pmf_error
#            best_results['best_params0'] = best_params0
#            best_results['best_params'] = best_params
#            np.save(save_file, best_results)

